## Cats and Dogs Mask Segmentation using tf.data

Forked from: https://www.tensorflow.org/tutorials/images/segmentation

At the end there is alternate model training example.

In [0]:
#"""
# Google Collab specific stuff....
from google.colab import drive
drive.mount('/content/drive')

import os
!ls "/content/drive/My Drive"

USING_COLLAB = True
%tensorflow_version 2.x
#"""

In [0]:
# Setup sys.path to find MachineLearning lib directory

try: USING_COLLAB
except NameError: USING_COLLAB = False

%load_ext autoreload
%autoreload 2

import sys
if "MachineLearning" in sys.path[0]:
    pass
else:
    print(sys.path)
    if USING_COLLAB:
        sys.path.insert(0, '/content/drive/My Drive/GitHub/MachineLearning/lib')  ###### CHANGE FOR SPECIFIC ENVIRONMENT
    else:
        sys.path.insert(0, '/Users/john/Documents/GitHub/MachineLearning/lib')  ###### CHANGE FOR SPECIFIC ENVIRONMENT
    
    print(sys.path)

In [0]:
!pip install -q git+https://github.com/tensorflow/examples.git

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os, sys, random, warnings, time, copy, csv, gc
import numpy as np 

import IPython.display as display
from IPython.display import clear_output

from PIL import Image
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow_examples.models.pix2pix import pix2pix
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from tqdm import tqdm_notebook, tnrange, tqdm

import tensorflow as tf
print(tf.__version__)

AUTOTUNE = tf.data.experimental.AUTOTUNE
print("AUTOTUNE: ", AUTOTUNE)

from TrainingUtils import *

#warnings.filterwarnings("ignore", category=DeprecationWarning)
#warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", "(Possibly )?corrupt EXIF data", UserWarning)

## Examine and understand data


In [0]:
# GLOBALS/CONFIG ITEMS

# Set root directory path to data
if USING_COLLAB:
    ROOT_PATH = "/content/drive/My Drive/ImageData/DogsCats"  ###### CHANGE FOR SPECIFIC ENVIRONMENT
else:
    ROOT_PATH = "/Users/john/Documents/ImageData/DogsCats"  ###### CHANGE FOR SPECIFIC ENVIRONMENT
        
# Establish global dictionary
parms = GlobalParms(MODEL_NAME="model-CatsDogs-masks-transfer-V01.h5",
                    ROOT_PATH=ROOT_PATH,
                    TRAIN_DIR="train", 
                    SMALL_RUN=False,
                    NUM_CLASSES=3,
                    IMAGE_ROWS=224,
                    IMAGE_COLS=224,
                    IMAGE_CHANNELS=3,
                    BATCH_SIZE=64,
                    EPOCS=10,  # set larger to improve results
                    IMAGE_EXT=".jpg",
                    LOSS=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    METRICS=['accuracy'])

parms.print_contents()

In [0]:

def show_batch_mask(display_list):
    plt.figure(figsize=(15, 15))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()

In [0]:
# Download dataset to local VM
builder = tfds.builder('oxford_iiit_pet:3.1.0')
info = builder.info

# by setting register_checksums as True to pass the check
config = tfds.download.DownloadConfig(register_checksums = True)
builder.download_and_prepare(download_config=config)
full_dataset = builder.as_dataset()

print(info)

In [0]:
# Download dataset to local VM
#full_dataset, info = tfds.load('oxford_iiit_pet:3.1.0', with_info=True)
#print(info)

In [0]:
# Set Class names...
parms.set_class_names(["Pet", "Bordering", "None"])
print("Classes: ", parms.NUM_CLASSES, 
      "   Labels: ", len(parms.CLASS_NAMES), 
      "  ", parms.CLASS_NAMES)


In [0]:
!ls "/root/tensorflow_datasets"

## Build an input pipeline

In [0]:
def normalize(image: tf.Tensor, mask: tf.Tensor) -> tf.Tensor:
    image = tf.cast(image, tf.float32) / 255.0
    mask -= 1
    return image, mask

def process_train(cat_dog_dict: tf.Tensor) -> tf.Tensor:
    image = tf.image.resize(cat_dog_dict['image'], (parms.IMAGE_ROWS, parms.IMAGE_COLS))
    mask = tf.image.resize(cat_dog_dict['segmentation_mask'], (parms.IMAGE_ROWS, parms.IMAGE_COLS))
  
    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)
        mask = tf.image.flip_left_right(mask)

    image, mask = normalize(image, mask)
    return image, mask

def process_val(cat_dog_dict: tf.Tensor) -> tf.Tensor:
    image = tf.image.resize(cat_dog_dict['image'], (parms.IMAGE_ROWS, parms.IMAGE_COLS))
    mask = tf.image.resize(cat_dog_dict['segmentation_mask'], (parms.IMAGE_ROWS, parms.IMAGE_COLS))

    image, mask = normalize(image, mask)
    return image, mask

In [0]:
#Set buffer size
BUFFER_SIZE = 1000

# set datasets
train_dataset = full_dataset["train"]
val_dataset = full_dataset["test"]

# set lengths
train_len = info.splits['train'].num_examples
val_len = info.splits['test'].num_examples
images_list_len = train_len + val_len

steps_per_epoch = np.ceil(train_len // parms.BATCH_SIZE) # set step sizes based on train & batch
validation_steps = np.ceil(val_len // parms.BATCH_SIZE) # set step sizes based on val & batch

print("Total number: ", images_list_len, "  Train number: ", train_len, "  Val number: ", val_len)
print("Steps/EPOC: ", steps_per_epoch, "  Steps/Validation: ", validation_steps)

### Training setup

In [0]:
# Verify image paths were loaded and save one path for later in "some_image"
for f in train_dataset.take(2):
    some_image = f["image"]
    some_mask = f["segmentation_mask"]
    print(some_mask.shape)

# map training images to pre-cache processing, includes any augmentation
train_dataset = train_dataset.map(process_train, num_parallel_calls=AUTOTUNE)

# Verify the mapping worked
for image, mask in train_dataset.take(1):
    print("Image shape: {}  Max: {}  Min: {}".format(image.numpy().shape, np.max(image.numpy()), np.min(image.numpy())))
    print("segmentation_mask: ", mask.shape, np.max(mask.numpy()), np.min(mask.numpy()))

#train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(parms.BATCH_SIZE).repeat()
train_dataset = train_dataset.cache().shuffle(BUFFER_SIZE).batch(parms.BATCH_SIZE).repeat()


In [0]:
# Show the images, execute this cell multiple times to see the images

for image, mask in train_dataset.take(1):
    sample_image, sample_mask = image[0], mask[0]
show_batch_mask([sample_image, sample_mask])

### Validation setup

In [0]:
# Verify image paths were loaded and save one path for later in "some_image"
for f in val_dataset.take(2):
    some_mask = f["segmentation_mask"]
    print(some_mask.shape)

# map training images to processing, includes any augmentation
val_dataset = val_dataset.map(process_val, num_parallel_calls=AUTOTUNE)

# Verify the mapping worked
for image, mask in val_dataset.take(1):
    print("Image shape: {}  Max: {}  Min: {}".format(image.numpy().shape, np.max(image.numpy()), np.min(image.numpy())))
    print("segmentation_mask: ", mask.shape)

val_dataset = val_dataset.cache().batch(parms.BATCH_SIZE).repeat()
#val_dataset = val_dataset.batch(parms.BATCH_SIZE).repeat()


In [0]:
# Test Validation, use smaller "number_to_show" to help show the augmentation
for image_batch, mask_batch in val_dataset.take(1):
    sample_image, sample_mask = image_batch[0], mask_batch[0]
show_batch_mask([sample_image, sample_mask])

## Build  model
- add and validate pretrained model as a baseline

In [0]:
class DisplayCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        clear_output(wait=True)
        show_predictions()
        print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, min_lr=1e-6)
earlystopper = EarlyStopping(patience=8, verbose=1)
checkpointer = ModelCheckpoint(parms.MODEL_PATH, monitor='val_loss', verbose=1, mode="auto", save_best_only=True)
   

In [0]:
base_model = tf.keras.applications.MobileNetV2(input_shape=[parms.IMAGE_ROWS, parms.IMAGE_COLS, 3], include_top=False)

# Use the activations of these layers
layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project'      # 4x4
]
layers = [base_model.get_layer(name).output for name in layer_names]

# Create the feature extraction model
down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)
down_stack.trainable = False

up_stack = [
    pix2pix.upsample(512, 3),  # 4x4 -> 8x8
    pix2pix.upsample(256, 3),  # 8x8 -> 16x16
    pix2pix.upsample(128, 3),  # 16x16 -> 32x32
    pix2pix.upsample(64, 3),   # 32x32 -> 64x64
]


def create_mask(pred_mask):
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]


def show_predictions(dataset=None, num=1):
    if dataset:
        for image, mask in dataset.take(num):
            pred_mask = model.predict(image)
            show_batch_mask([image[0], mask[0], create_mask(pred_mask)])
    else:
        show_batch_mask([sample_image, sample_mask,
             create_mask(model.predict(sample_image[tf.newaxis, ...]))])
    
def build_unet_model(output_channels):
    inputs = tf.keras.layers.Input(shape=[parms.IMAGE_ROWS, parms.IMAGE_COLS, 3])
    x = inputs

    # Downsampling through the model
    skips = down_stack(x)
    x = skips[-1]
    skips = reversed(skips[:-1])

    # Upsampling and establishing the skip connections
    for up, skip in zip(up_stack, skips):
        x = up(x)
        #####
        x = tf.keras.layers.BatchNormalization()(x)
        #####
        concat = tf.keras.layers.Concatenate()
        x = concat([x, skip])

    # This is the last layer of the model
    last = tf.keras.layers.Conv2DTranspose(
        output_channels, 3, strides=2,
        padding='same')  #64x64 -> 128x128

    x = last(x)

    return tf.keras.Model(inputs=inputs, outputs=x)

def compile_model(parms, model):
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=parms.LOSS,
                  metrics=parms.METRICS)
    return model


## Train model

In [0]:
# Train model
model = build_unet_model(parms.NUM_CLASSES)
model = compile_model(parms, model)

In [0]:
#model.save(parms.ROOT_PATH+ "/" + "startModel.h5")
#print(parms.ROOT_PATH+ "/" + "startModel.h5")

In [0]:
show_predictions()

In [0]:
history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=parms.EPOCS, 
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps,
                    callbacks=[DisplayCallback(), reduce_lr, earlystopper])


In [0]:

# Plot the training history
history_df = pd.DataFrame(history.history)
plt.figure()
history_df[['loss', 'val_loss']].plot(title="Loss")
plt.xlabel('Epocs')
plt.ylabel('Loss')
history_df[['accuracy', 'val_accuracy']].plot(title="Accuracy")
plt.xlabel('Epocs')
plt.ylabel('Accuracy')
plt.show()

In [0]:
show_predictions(val_dataset, 3)

## Alternate model training approach


In [0]:
#https://www.kaggle.com/yushas/imageprocessingtips
#Definie UNet Builder
def conv_block_mod(m, dim, acti, bn, res, do=0):
    n = tf.keras.layers.Conv2D(dim, 3, activation=acti, padding='same')(m)
    n = tf.keras.layers.BatchNormalization()(n) if bn else n
    n = tf.keras.layers.Dropout(do)(n) if do else n
    n = tf.keras.layers.Conv2D(dim, 3, activation=acti, padding='same')(n)
    n = tf.keras.layers.BatchNormalization()(n) if bn else n
    return tf.keras.layers.Concatenate()([m, n]) if res else n

def level_block_mod(m, dim, depth, inc, acti, do, bn, mp, up, res):
    if depth > 0:
        n = conv_block_mod(m, dim, acti, bn, res)
        m = tf.keras.layers.MaxPooling2D()(n) if mp else Conv2D(dim, 3, strides=2, padding='same')(n)
        m = level_block_mod(m, int(inc*dim), depth-1, inc, acti, do, bn, mp, up, res)#再帰
        if up:
            m = tf.keras.layers.UpSampling2D()(m)
            m = tf.keras.layers.Conv2D(dim, 2, activation=acti, padding='same')(m)
        else:
            m = tf.keras.layers.Conv2DTranspose(dim, 3, strides=2, activation=acti, padding='same')(m)
        n = tf.keras.layers.Concatenate()([n, m])
        m = conv_block_mod(n, dim, acti, bn, res)
    else:
        m = conv_block_mod(m, dim, acti, bn, res, do)
    return m

def UNet_mod(img_shape, out_ch=1, start_ch=64, depth=4, inc_rate=2., activation='relu', 
         dropout=False, batchnorm=True, maxpool=True, upconv=True, residual=False):
    i = tf.keras.layers.Input(shape=img_shape)
    o = level_block_mod(i, start_ch, depth, inc_rate, activation, dropout, batchnorm, maxpool, upconv, residual)#Unet
    o = tf.keras.layers.Conv2D(out_ch, 1, activation='sigmoid')(o)
    return Model(inputs=i, outputs=o)


In [0]:
class DisplayCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        clear_output(wait=True)
        show_predictions()
        print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, min_lr=1e-6)
earlystopper = EarlyStopping(patience=8, verbose=1)
   

In [0]:
model2 = UNet_mod((parms.IMAGE_ROWS, parms.IMAGE_COLS, 3), out_ch=3, start_ch=16,depth=4,batchnorm=True,dropout=0.5)
model2 = compile_model(parms, model2)

In [0]:
history2 = model2.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=parms.EPOCS, 
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps,
                    callbacks=[DisplayCallback(), reduce_lr, earlystopper])


In [0]:

# Plot the training history
history_df = pd.DataFrame(history2.history)
plt.figure()
history_df[['loss', 'val_loss']].plot(title="Loss")
plt.xlabel('Epocs')
plt.ylabel('Loss')
history_df[['accuracy', 'val_accuracy']].plot(title="Accuracy")
plt.xlabel('Epocs')
plt.ylabel('Accuracy')
plt.show()

In [0]:
for image, mask in val_dataset.take(5):
    pred_mask = model2.predict(image)
    show_batch_mask([image[0], mask[0], create_mask(pred_mask)])
